In [1]:
import pandas as pd
import os
import glob
import csv
import datetime
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas_ta as ta
import vectorbtpro as vbt

In [65]:
def rename_column(df):
    new_column_names = ['open_time','open','high','low','close','volume','close_time','quote_volume','count','taker_buy_volume','taker_buy_quote_volume','ignore']
    df.columns = new_column_names
    return df

In [66]:
def epoch_to_datetime(df):
    df['datetime'] =  df['open_time'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000.0))
    return df

In [2]:
def hma(df,period):
    wma_1 = df['Close'].rolling(period//2).apply(lambda x: \
    np.sum(x * np.arange(1, period//2+1)) / np.sum(np.arange(1, period//2+1)), raw=True)
    wma_2 = df['Close'].rolling(period).apply(lambda x: \
    np.sum(x * np.arange(1, period+1)) / np.sum(np.arange(1, period+1)), raw=True)
    diff = 2 * wma_1 - wma_2
    hma = diff.rolling(int(np.sqrt(period))).mean()
    return hma

In [3]:
df_4h = pd.read_csv('/Users/panthira/kedro-backtest/kedro-supertrend/data/02_intermediate/data_processed_4h.csv')
df_1d = pd.read_csv('/Users/panthira/kedro-backtest/kedro-supertrend/data/02_intermediate/data_processed_1d.csv')

In [4]:
def Supertrend(df, atr_period, multiplier):  # 10,3
    high = df['High']
    low = df['Low']
    close = df['Close']
    
    # calculate ATR
    price_diffs = [high - low, 
                    high - close.shift(), 
                    close.shift() - low]
    true_range = pd.concat(price_diffs, axis=1)
    true_range = true_range.abs().max(axis=1)
    # default ATR calculation in supertrend indicator
    atr = true_range.ewm(alpha=1/atr_period,min_periods=atr_period).mean() 
    # df['atr'] = df['tr'].rolling(atr_period).mean()
    
    # HL2 is simply the average of high and low prices
    hl2 = hma(df,100)
    # hl2 = (high + low) / 2
    # upperband and lowerband calculation
    # notice that final bands are set to be equal to the respective bands
    final_upperband = upperband = hl2 + (multiplier * atr)
    final_lowerband = lowerband = hl2 - (multiplier * atr)
    
    # initialize Supertrend column to True
    supertrend = [True] * len(df)
    
    for i in range(1, len(df.index)):
        curr, prev = i, i-1
        
        # if current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True
        # if current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False
        # else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]
            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

        # to remove bands according to the trend direction
        if supertrend[curr] == True:
            final_upperband[curr] = np.nan
        else:
            final_lowerband[curr] = np.nan
    
    return pd.DataFrame({
        'Supertrend': supertrend,
        'Final Lowerband': final_lowerband,
        'Final Upperband': final_upperband
    }, index=df.index) 

In [5]:
st_4h = Supertrend(df_4h, 10, 0.5)
df_4h['lower'] = st_4h['Final Lowerband']
df_4h['upper'] = st_4h['Final Upperband']
df_4h['supertrend'] = st_4h['Supertrend']

st_1d = Supertrend(df_1d, 10, 0.5)
df_1d['lower'] = st_1d['Final Lowerband']
df_1d['upper'] = st_1d['Final Upperband']
df_1d['supertrend'] = st_1d['Supertrend']

In [6]:
df_4h['Datetime'] = pd.to_datetime(df_4h['Datetime'])
df_4h.set_index('Datetime', inplace=True)

df_1d['Datetime'] = pd.to_datetime(df_1d['Datetime'])
df_1d.set_index('Datetime', inplace=True)

In [7]:
def hma(df,period):
    wma_1 = df['Close'].rolling(period//2).apply(lambda x: \
    np.sum(x * np.arange(1, period//2+1)) / np.sum(np.arange(1, period//2+1)), raw=True)
    wma_2 = df['Close'].rolling(period).apply(lambda x: \
    np.sum(x * np.arange(1, period+1)) / np.sum(np.arange(1, period+1)), raw=True)
    diff = 2 * wma_1 - wma_2
    hma = diff.rolling(int(np.sqrt(period))).mean()
    return hma


In [8]:
df_4h['hma'] = hma(df_4h, 100)
df_1d['hma'] = hma(df_1d, 100)

In [9]:
def fillcol(label):
    if label >= 1:
        return 'rgba(0,250,0,0.4)'
    else:
        return 'rgba(250,0,0,0.4)'

In [34]:
triangle = np.where(df_4h['buy_sell_signal']== -1, 'triangle-down',0)
df_4h["Symbol"] = np.where(df_4h["buy_sell_signal"]==1, "triangle-up", triangle)
color = np.where(df_4h['buy_sell_signal']== -1, 'red',0)
df_4h["Color"] = np.where(df_4h["buy_sell_signal"]==1, "green", color)

df_buy = df_4h[df_4h['buy_sell_signal']==1]
df_sell = df_4h[df_4h['buy_sell_signal']==-1]
fig = make_subplots(rows=1, cols=1)

# fig.add_trace(
#     go.Candlestick(x=df['Datetime'], 
#                     open=df['Open'], 
#                     high=df['High'], 
#                     low=df['Low'], 
#                     close=df['Close']), row=1, col=1
# )
fig.add_trace(
    go.Scatter(
        x=df_4h.index,
        y=df_4h['Close'],
        line=dict(color='Black', width=2),
        name='hma',
        legendgroup='1',
    ), row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df_buy.index,
        y=df_buy['Close'],
        mode='markers',
        name ='markers',
        marker=go.Marker(size=10,
                        symbol=df_buy["Symbol"],
                        color=df_buy["Color"])
    ))

fig.add_trace(
    go.Scatter(
        x=df_sell.index,
        y=df_sell['Close'],
        mode='markers',
        name ='markers',
        marker=go.Marker(size=10,
                        symbol=df_sell["Symbol"],
                        color=df_sell["Color"])
    ))


fig.add_trace(
    go.Scatter(
        x=df_4h.index,
        y=df_4h['lower+upper_1d'],
        line=dict(color='grey', width=2),
        name='st_ma_1d',
        legendgroup='1',
    ), row=1, col=1
)

fig.add_scattergl(x = df_4h.index,
                    y = df_4h['lower+upper_1d'].where(df_4h['supertrend_1d']==True),
                    line={'color':'green'})


fig.add_scattergl(x = df_4h.index,
                    y = df_4h['lower+upper_1d'].where(df_4h['supertrend_1d']==False),
                    line={'color':'red'})


# fig.add_trace(
#     go.Scatter(
#         x=df_1d.index,
#         y=df_1d['lower'],
#         line=dict(color='Green', width=2),
#         name='lower',
#         legendgroup='1',
#     ), row=1, col=1
# )

# fig.add_trace(
#     go.Scatter(
#         x=df_1d.index,
#         y=df_1d['upper'],
#         line=dict(color='Red', width=2),
#         name='upper',
#         legendgroup='1',
#     ), row=1, col=1
# )

# fig.add_trace(
#     go.Scatter(
#         x=df_4h.index,
#         y=df_4h['hma'],
#         line=dict(color='Pink', width=1),
#         name='1d',
#         legendgroup='1',
#     ), row=1, col=1
# )

fig.add_trace(
    go.Scatter(
        x=df_4h.index,
        y=df_4h['lower+upper'],
        line=dict(color='grey', width=2),
        name='lower',
        legendgroup='1',
        # fill='tonexty', 
        # fillcolor = fillcol(df_1h['label'].iloc[0])
    ), row=1, col=1
)

fig.add_scattergl(x = df_4h.index,
                    y = df_4h['lower+upper'].where(df_4h['supertrend']==True),
                    line={'color':'green'})


fig.add_scattergl(x = df_4h.index,
                    y = df_4h['lower+upper'].where(df_4h['supertrend']==False),
                    line={'color':'red'})


# fig.add_trace(
#     go.Scatter(
#         x=df_1h.index,
#         y=df_1h['upper'],
#         line=dict(color='Red', width=1),
#         name='upper',
#         legendgroup='1',
#         fill='tonexty', 
#         fillcolor = fillcol(df['label'].iloc[0])
#     ), row=1, col=1
# )



fig.update_layout(xaxis_rangeslider_visible = False, title = 'OPUSDT', width = 1400, height = 500 )
fig.update_xaxes(title_text = 'Date')
fig.update_yaxes(title_text = 'OPUSDT Close Price', tickprefix = '$')

fig.show()

In [12]:
df_4h = df_4h.fillna(0)
df_4h['lower+upper'] = df_4h['lower'] + df_4h['upper']
df_4h.replace(0, np.nan, inplace=True)

In [13]:
df_1d = df_1d.fillna(0)
df_1d['lower+upper'] = df_1d['lower'] + df_1d['upper']
df_1d.replace(0, np.nan, inplace=True)

In [14]:
df_4h['label'] = np.where(df_4h['supertrend']==True, 1, 0)

In [15]:
import pandas as pd

# Assuming both DataFrames have datetime indices
df_4h['date'] = df_4h.index.date
df_1d['date'] = df_1d.index.date

# Merge the DataFrames based on the date component
merged_df = df_4h.merge(df_1d[['date', 'supertrend']], on='date', how='left')
supertrend_1d = merged_df['supertrend_y'].to_list()

merged_df = df_4h.merge(df_1d[['date', 'lower+upper']], on='date', how='left')

lower_upper_1d = merged_df['lower+upper_y'].to_list()

df_4h['lower+upper_1d'] = lower_upper_1d
df_4h['supertrend_1d'] = supertrend_1d
df_4h.drop(columns=['date'], inplace=True)

In [16]:
df_4h

,Open,High,Low,Close,Volume,lower,upper,supertrend,hma,lower+upper,label,lower+upper_1d,supertrend_1d
Datetime,,,,,,,,,,,,,
2022-08-01 11:00:00,1.649,1.655,1.580,1.626,4388808.79,NaN,NaN,True,NaN,NaN,1,NaN,NaN
2022-08-01 15:00:00,1.627,1.678,1.533,1.581,7522756.04,NaN,NaN,True,NaN,NaN,1,NaN,NaN
2022-08-01 19:00:00,1.580,1.690,1.558,1.644,10153605.06,NaN,NaN,True,NaN,NaN,1,NaN,NaN
2022-08-01 23:00:00,1.643,1.646,1.427,1.475,11772247.43,NaN,NaN,True,NaN,NaN,1,NaN,NaN
2022-08-02 03:00:00,1.474,1.529,1.436,1.509,8126824.26,NaN,NaN,True,NaN,NaN,1,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-31 11:00:00,1.678,1.695,1.665,1.689,5480280.73,1.487954,NaN,True,1.509964,1.487954,1,1.261984,True
2023-07-31 15:00:00,1.689,1.771,1.675,1.686,10485413.12,1.489455,NaN,True,1.514724,1.489455,1,1.261984,True
2023-07-31 19:00:00,1.684,1.694,1.644,1.646,12343626.39,1.494995,NaN,True,1.520238,1.494995,1,1.261984,True


In [24]:
df_4h['supertrend_1d_shift_1'] = df_4h['supertrend_1d'].shift(1)
df_4h['supertrend_shift_1'] = df_4h['supertrend'].shift(1)

In [679]:
df_4h['is_highest'] = np.where(df_4h['Close_shift_1'] == df_4h['Close_shift_1'].rolling(30).max(), True, False)
df_4h['is_lowest'] = np.where(df_4h['Close_shift_1'] == df_4h['Close_shift_1'].rolling(30).min(), True, False)

In [26]:
df_4h['rsi'] = vbt.RSI.run(df_4h['Close'], window=7).rsi

In [51]:
df_4h['macd_hist'] = vbt.MACD.run(df_4h['Close']).hist

Condition


In [41]:
triangle = np.where(df_4h['buy_sell_signal']== -1, 'triangle-down',0)
df_4h["Symbol"] = np.where(df_4h["buy_sell_signal"]==1, "triangle-up", triangle)
color = np.where(df_4h['buy_sell_signal']== -1, 'red',0)
df_4h["Color"] = np.where(df_4h["buy_sell_signal"]==1, "green", color)

df_buy = df_4h[df_4h['buy_sell_signal']==1]
df_sell = df_4h[df_4h['buy_sell_signal']==-1]

fig = make_subplots(rows=2, cols=1, shared_xaxes=True)


fig.add_trace(
    go.Scatter(
        x=df_4h.index,
        y=df_4h['Close'],
        line=dict(color='Black', width=2),
        name='hma',
        legendgroup='1',
    ), row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df_buy.index,
        y=df_buy['Close'],
        mode='markers',
        name ='markers',
        marker=go.Marker(size=10,
                        symbol=df_buy["Symbol"],
                        color=df_buy["Color"])
    ))

fig.add_trace(
    go.Scatter(
        x=df_sell.index,
        y=df_sell['Close'],
        mode='markers',
        name ='markers',
        marker=go.Marker(size=10,
                        symbol=df_sell["Symbol"],
                        color=df_sell["Color"])
    ))


# fig.add_trace(
#     go.Scatter(
#         x=df_4h.index,
#         y=df_4h['lower+upper_1d'],
#         line=dict(color='grey', width=2),
#         name='st_ma_1d',
#         legendgroup='1',
#     ), row=1, col=1
# )

# fig.add_scattergl(x = df_4h.index,
#                     y = df_4h['lower+upper_1d'].where(df_4h['supertrend_1d']==True),
#                     line={'color':'green'})


# fig.add_scattergl(x = df_4h.index,
#                     y = df_4h['lower+upper_1d'].where(df_4h['supertrend_1d']==False),
#                     line={'color':'red'})


fig.add_trace(
    go.Scatter(
        x=df_4h.index,
        y=df_4h['lower+upper'],
        line=dict(color='grey', width=2),
        name='lower',
        legendgroup='1',
        # fill='tonexty', 
        # fillcolor = fillcol(df_1h['label'].iloc[0])
    ), row=1, col=1
)

fig.add_scattergl(x = df_4h.index,
                    y = df_4h['lower+upper'].where(df_4h['supertrend']==True),
                    line={'color':'green'})


fig.add_scattergl(x = df_4h.index,
                    y = df_4h['lower+upper'].where(df_4h['supertrend']==False),
                    line={'color':'red'})





# fig.add_trace(
#     go.Scatter(
#         x=df_1h.index,
#         y=df_1h['upper'],
#         line=dict(color='Red', width=1),
#         name='upper',
#         legendgroup='1',
#         fill='tonexty', 
#         fillcolor = fillcol(df['label'].iloc[0])
#     ), row=1, col=1
# )


# fig.add_trace(
#     go.Scatter(
#         x=df_4h.index,
#         y=df_4h['rsi'],
#         line=dict(color='blue', width=1),
#         name='hma',
#         legendgroup='1',
#     ), row=2, col=1
# )

# fig.add_trace(
#     go.Scatter(
#         x=df_4h.index,
#         y= [60] * len(df_4h.index),
#         line=dict(color='grey', width=1),
#         name='rsi',
#         legendgroup='1',
#     ), row=2, col=1
# )

# fig.add_trace(
#     go.Scatter(
#         x=df_4h.index,
#         y= [40] * len(df_4h.index),
#         line=dict(color='grey', width=1),
#         name='rsi',
#         legendgroup='1',
#     ), row=2, col=1
# )

colors = np.where(df['macdh_12_26_9'] < 0, '#000', '#ff9900')
fig.add_trace(
    go.Bar(
        x=df_4h.index,
        y=df_4h['macd_hist'],
        name='histogram',
        marker_color=colors,
    ), row=2, col=1
)

fig.update_layout(xaxis_rangeslider_visible = False, title = 'OPUSDT', width = 1400, height = 800 )
fig.update_xaxes(title_text = 'Date')
fig.update_yaxes(title_text = 'OPUSDT Close Price')

fig.update_xaxes(showgrid=False, zeroline=False, rangeslider_visible=False, showticklabels=False,
                 showspikes=True, spikemode='across', spikesnap='cursor', showline=False,
                 spikecolor="grey",spikethickness=1, spikedash='solid')

fig.update_layout(spikedistance=1000,hoverdistance=1000)


fig.show()

In [1132]:
df_4h['cross_above_60'] = df_4h['rsi'].vbt.crossed_below(60) 
df_4h['cross_above_40'] = df_4h['rsi'].vbt.crossed_above(30) 

df_4h[df_4h['cross_above_60'] == True]
df_4h[df_4h['cross_above_40'] == True]

,Open,High,Low,Close,Volume,lower,upper,supertrend,hma,lower+upper,...,supertrend_shift_20,supertrend_1d_shift_120,supertrend_1d_shift_15,supertrend_1d_shift_30,supertrend_shift_28,supertrend_shift_60,supertrend_shift_120,rsi,cross_above_60,cross_above_40
Datetime,,,,,,,,,,,,,,,,,,,,,
2022-08-10 07:00:00,1.616,1.716,1.595,1.679,14547068.97,NaN,NaN,True,NaN,NaN,...,True,None,True,True,True,NaN,NaN,31.462175,False,True
2022-08-11 07:00:00,1.569,1.674,1.568,1.665,14410611.02,NaN,NaN,True,NaN,NaN,...,True,None,True,True,True,NaN,NaN,43.244561,False,True
2022-08-12 03:00:00,1.521,1.591,1.490,1.554,6642309.44,NaN,NaN,True,NaN,NaN,...,True,None,True,True,True,True,NaN,34.728543,False,True
2022-08-15 07:00:00,1.438,1.541,1.407,1.507,13379219.25,NaN,NaN,True,NaN,NaN,...,True,None,True,True,True,True,NaN,45.461140,False,True
2022-08-15 15:00:00,1.388,1.454,1.386,1.435,7041161.16,NaN,NaN,True,NaN,NaN,...,True,None,True,True,True,True,NaN,38.985705,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-07 07:00:00,1.185,1.210,1.169,1.204,2221581.53,NaN,1.307822,False,1.289308,1.307822,...,True,False,True,True,True,False,True,32.976648,False,True
2023-07-07 15:00:00,1.192,1.210,1.187,1.202,2075505.96,NaN,1.298437,False,1.281390,1.298437,...,True,False,True,True,True,False,True,35.974724,False,True
2023-07-10 15:00:00,1.181,1.190,1.155,1.186,3387327.80,NaN,1.227478,False,1.212671,1.227478,...,False,True,True,True,False,False,True,32.159391,False,True


In [28]:
df_4h['buy_signal'] = np.where(
    (
        (
            (df_4h['supertrend'] == True) &
            (df_4h['supertrend_shift_1'] == False) 
        )
        
    ),
    1,
    0
)

df_4h['sell_signal'] = np.where(
    (
        (
            (df_4h['supertrend'] == False) &
            (df_4h['supertrend_shift_1'] == True) 
        )
        
    ),
    1,
    0
)


In [1192]:
df_4h['buy_signal'] = np.where(
    (
        (
            (df_4h['supertrend_1d'] == False) &
            (df_4h['supertrend_1d_shift_1'] == True) &
            (df_4h['cross_above_40'] == True)
        )
        |
        (
            (df_4h['supertrend'] == False) &
            (df_4h['supertrend_shift_1'] == True)
        )
        |
        (
            (df_4h['supertrend'] == True) &
            (df_4h['supertrend_1d'] == True) 
        )
        |
        (
            (df_4h['supertrend_shift_1'] == False) &
            (df_4h['supertrend'] == True) &
            (df_4h['supertrend_1d'] == True) 
        ) |
        (
            (df_4h['is_lowest'] == True) 

        )
    ),
    1,
    0
)


In [1191]:
df_4h['sell_signal'] = np.where(
    (
        (
            (df_4h['supertrend_1d_shift_1'] == True) &
            (df_4h['supertrend_1d'] == False) 
        )
        |
        (
            (df_4h['supertrend'] == False) &
            (df_4h['supertrend_shift_1'] == True)
        )
        |
        (
            (df_4h['supertrend'] == False) &
            (df_4h['supertrend_1d'] == False)
        )
        |
        (
            (df_4h['supertrend_shift_1'] == True) &
            (df_4h['supertrend'] == False) &
            (df_4h['supertrend_1d'] == False)
        ) |
        (
            (df_4h['is_highest'] == True) &
            (df_4h['supertrend_shift_1'] == True)

        )
    ),
    1,
    0
)


In [29]:
df_4h['buy_sell_signal'] = np.where(df_4h['buy_signal'] == 1, 1, np.where(df_4h['sell_signal'] == 1, -1, 0))

In [741]:
import vectorbtpro as vbt

In [1195]:
entries.value_counts()


False    1159
True     1019
Name: entries, dtype: int64

In [43]:
df_4h['entries'] = np.where(df_4h['buy_sell_signal']== 1 , True,False)
df_4h['exits'] = np.where(df_4h['buy_sell_signal']== -1 , True,False)

entries = df_4h['entries']
exits = df_4h['exits']

In [44]:
clean_entries, clean_exits = entries.vbt.signals.clean(exits)
print(clean_entries.vbt.signals.total())
print(clean_exits.vbt.signals.total() )

46
45


In [45]:
ranges = clean_entries.vbt.signals.between_ranges(other=clean_exits)
ranges.duration.mean(wrap_kwargs=dict(to_timedelta=True)) 

[09/19/23 15:05:13] WARNING  /opt/anaconda3/envs/kedro-vbt/lib/python3.10/site-packages/vectorbtpro ]8;id=57928;file:///opt/anaconda3/envs/kedro-vbt/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=784667;file:///opt/anaconda3/envs/kedro-vbt/lib/python3.10/warnings.py#109\109]8;;\
                             /base/wrapping.py:968: UserWarning:                                                   
                                                                                                                   
                             Couldn't parse the frequency of index. Pass it as `freq` or define it                 
                             globally under `settings.wrapping`.                                                   
                                                                                                                   
                                                                                                                   

23.466666666666665

In [46]:
pf = vbt.Portfolio.from_signals(
    close=df_4h['Close'], 
    entries=entries, 
    exits=exits,
    size=100,
    size_type='value',
    init_cash='auto'
)
pf.stats()


[09/19/23 15:05:33] WARNING  /opt/anaconda3/envs/kedro-vbt/lib/python3.10/site-packages/vectorbtpro ]8;id=603071;file:///opt/anaconda3/envs/kedro-vbt/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=130461;file:///opt/anaconda3/envs/kedro-vbt/lib/python3.10/warnings.py#109\109]8;;\
                             /generic/stats_builder.py:463: UserWarning:                                           
                                                                                                                   
                             Metric 'sharpe_ratio' requires frequency to be set                                    
                                                                                                                   
                                                                                                                   

                    WARNING  /opt/anaconda3/envs/kedro-vbt/lib/python3.10/site-packages/vectorbtpro ]8;id=495041;file:///opt/anaconda3/envs/kedro-vbt/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=505546;file:///opt/anaconda3/envs/kedro-vbt/lib/python3.10/warnings.py#109\109]8;;\
                             /generic/stats_builder.py:463: UserWarning:                                           
                                                                                                                   
                             Metric 'calmar_ratio' requires frequency to be set                                    
                                                                                                                   
                                                                                                                   

                    WARNING  /opt/anaconda3/envs/kedro-vbt/lib/python3.10/site-packages/vectorbtpro ]8;id=283481;file:///opt/anaconda3/envs/kedro-vbt/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=355268;file:///opt/anaconda3/envs/kedro-vbt/lib/python3.10/warnings.py#109\109]8;;\
                             /generic/stats_builder.py:463: UserWarning:                                           
                                                                                                                   
                             Metric 'omega_ratio' requires frequency to be set                                     
                                                                                                                   
                                                                                                                   

                    WARNING  /opt/anaconda3/envs/kedro-vbt/lib/python3.10/site-packages/vectorbtpro ]8;id=412256;file:///opt/anaconda3/envs/kedro-vbt/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=784387;file:///opt/anaconda3/envs/kedro-vbt/lib/python3.10/warnings.py#109\109]8;;\
                             /generic/stats_builder.py:463: UserWarning:                                           
                                                                                                                   
                             Metric 'sortino_ratio' requires frequency to be set                                   
                                                                                                                   
                                                                                                                   

[09/19/23 15:05:37] WARNING  /opt/anaconda3/envs/kedro-vbt/lib/python3.10/site-packages/vectorbtpro ]8;id=843418;file:///opt/anaconda3/envs/kedro-vbt/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=467979;file:///opt/anaconda3/envs/kedro-vbt/lib/python3.10/warnings.py#109\109]8;;\
                             /base/wrapping.py:968: UserWarning:                                                   
                                                                                                                   
                             Couldn't parse the frequency of index. Pass it as `freq` or define it                 
                             globally under `settings.wrapping`.                                                   
                                                                                                                   
                                                                                                                   


Start                         2022-08-01 11:00:00
End                           2023-08-01 03:00:00
Period                                       2178
Start Value                            130.450033
Min Value                                   100.0
Max Value                              249.428209
End Value                              221.528397
Total Return [%]                        69.818583
Benchmark Return [%]                    -0.799508
Total Time Exposure [%]                 49.035813
Max Gross Exposure [%]                      100.0
Max Drawdown [%]                        30.879338
Max Drawdown Duration                       939.0
Total Orders                                   91
Total Fees Paid                               0.0
Total Trades                                   46
Win Rate [%]                            24.444444
Best Trade [%]                          87.783784
Worst Trade [%]                         -9.672387
Avg Winning Trade [%]                   21.849852

In [47]:
pf.plot(settings=dict(bm_returns=False)).show()

In [1213]:
df_4h

                    WARNING  /opt/anaconda3/envs/kedro-backtest/lib/python3.10/site-packages/IPytho ]8;id=620155;file:///opt/anaconda3/envs/kedro-backtest/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=291358;file:///opt/anaconda3/envs/kedro-backtest/lib/python3.10/warnings.py#109\109]8;;\
                             n/core/formatters.py:344: FutureWarning:                                              
                                                                                                                   
                             In a future version, object-dtype columns with all-bool values will                   
                             not be included in reductions with bool_only=True. Explicitly cast to                 
                             bool dtype instead.                                                                   
                                                                                                                   
                                                                                                                   

,Open,High,Low,Close,Volume,lower,upper,supertrend,hma,lower+upper,...,supertrend_shift_20,supertrend_1d_shift_120,supertrend_1d_shift_15,supertrend_1d_shift_30,supertrend_shift_28,supertrend_shift_60,supertrend_shift_120,rsi,cross_above_60,cross_above_40
Datetime,,,,,,,,,,,,,,,,,,,,,
2022-08-01 11:00:00,1.649,1.655,1.580,1.626,4388808.79,NaN,NaN,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
2022-08-01 15:00:00,1.627,1.678,1.533,1.581,7522756.04,NaN,NaN,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
2022-08-01 19:00:00,1.580,1.690,1.558,1.644,10153605.06,NaN,NaN,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
2022-08-01 23:00:00,1.643,1.646,1.427,1.475,11772247.43,NaN,NaN,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
2022-08-02 03:00:00,1.474,1.529,1.436,1.509,8126824.26,NaN,NaN,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-31 11:00:00,1.678,1.695,1.665,1.689,5480280.73,1.487954,NaN,True,1.509964,1.487954,...,False,True,True,True,False,False,True,83.771155,False,False
2023-07-31 15:00:00,1.689,1.771,1.675,1.686,10485413.12,1.489455,NaN,True,1.514724,1.489455,...,False,True,True,True,False,False,True,82.168532,False,False
2023-07-31 19:00:00,1.684,1.694,1.644,1.646,12343626.39,1.494995,NaN,True,1.520238,1.494995,...,False,True,True,True,False,False,True,63.323817,False,False


In [1214]:
df_1d

,Open,High,Low,Close,Volume,lower,upper,supertrend,hma,lower+upper,date
Datetime,,,,,,,,,,,
2022-08-02 07:00:00,1.509,1.671,1.362,1.636,7.876637e+07,NaN,NaN,True,NaN,NaN,2022-08-02
2022-08-03 07:00:00,1.635,2.191,1.540,1.991,1.188715e+08,NaN,NaN,True,NaN,NaN,2022-08-03
2022-08-04 07:00:00,1.991,2.243,1.824,1.918,1.085443e+08,NaN,NaN,True,NaN,NaN,2022-08-04
2022-08-05 07:00:00,1.917,2.082,1.796,1.943,7.905951e+07,NaN,NaN,True,NaN,NaN,2022-08-05
2022-08-06 07:00:00,1.946,2.055,1.870,1.900,5.127918e+07,NaN,NaN,True,NaN,NaN,2022-08-06
...,...,...,...,...,...,...,...,...,...,...,...
2023-07-27 07:00:00,1.512,1.569,1.490,1.497,1.547710e+07,1.186731,NaN,True,1.243875,1.186731,2023-07-27
2023-07-28 07:00:00,1.496,1.514,1.461,1.478,1.196822e+07,1.208487,NaN,True,1.262566,1.208487,2023-07-28
2023-07-29 07:00:00,1.477,1.516,1.473,1.509,7.602524e+06,1.229841,NaN,True,1.280661,1.229841,2023-07-29


In [6]:
import vectorbtpro as vbt